In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from bingX import BingX
import mplfinance as mpl
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import pandas_ta as ta

TRADE_INTERVALS = ["1M","1d","4h","1h","30m","15m","5m","3m","1m"]

# Exchange
API_KEY = os.environ.get("BINGX_API_KEY","")
SECRET_KEY = os.environ.get("BINGX_SECRET_KEY","")
bingx_client = BingX(API_KEY, SECRET_KEY)

def get_data(symbol, interval, start, end):
    start_time_ms = int(start.timestamp() * 1000)
    end_time_ms = int(end.timestamp() * 1000)
    data = bingx_client.perpetual_v2.market.get_k_line_data(symbol=symbol,interval=interval,start_time=start_time_ms,end_time=end_time_ms)
    df = pd.DataFrame(data)
    df.set_index('time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df


def get_levels(date):
    ratios = [-0.618, 0.618, 1.618] # SL - Entry - TP
    series_ = df.loc[date:][1:2].squeeze()
    diff = series_.high - series_.low
    levels = [series_.close + i * diff for i in ratios]
    return levels

def get_median_values(lf):
    n = 5
    lf['median'] = np.abs(lf['high'] + lf['low']) / 2
    lf['min'] = lf.iloc[argrelextrema(lf['median'].values, np.less_equal,
                                      order=n)[0]]['median']
    lf['max'] = lf.iloc[argrelextrema(lf['median'].values, np.greater_equal,
                                      order=n)[0]]['median']
    return lf

# Date intervals to fetch data
date = datetime(2022,1,1,0,0,0)
date_end = datetime(2022,1,5,0,0,0)
df = get_data("BTC-USDT", "1h", date, date_end)
lf = get_median_values(df)

#lf = df.drop(columns=['volume','open','close'])
#pdf = lf.drop(columns=['high','low'])
plt.scatter(lf.index, lf['min'], c='r')
for index, min_price in lf['min'].items():
    plt.text(index, min_price, f"{min_price:.2f}", fontsize=8, ha='right', va='bottom', color='r')
plt.scatter(lf.index, lf['max'], c='g')
for index, max_price in lf['max'].items():
    plt.text(index, max_price, f"{max_price:.2f}", fontsize=8, ha='right', va='bottom', color='g')
plt.plot(lf.index, lf['median'])
plt.show()

In [ ]:
min_max_vals = []
min_max_type = {0:[],1:[]}
filtered_df = lf[lf['min'].notnull() | lf['max'].notnull()]
def get_min_max_data():
    idx = 0
    while(idx < len(filtered_df)):
        row = filtered_df.iloc[idx]
        if not pd.isna(row['min']):
            min_max_vals.append(row['min'])
            min_max_type[0].append(row['min'])
        if not pd.isna(row['max']):
            min_max_vals.append(row['max'])
            min_max_type[1].append(row['max'])
        idx = idx + 1

def fibonacci_levels_v2(min_val, max_val):
    diff = max_val - min_val
    fib_5 = min_val + 0.5 * (diff)
    fib_618 = min_val + 0.618 * (diff)
    fib_786 = min_val + 0.786 * (diff)
    return fib_5, fib_618, fib_786

def possible_retracement(price, min_val, max_val):
    fib_5, fib_618, fib_786 = fibonacci_levels_v2(min_val, max_val)
    if fib_618 < price <= fib_786:
        print(f"The price {price} is within the range of 0.618 - 0.786 ({fib_618}, {fib_786})")
    elif fib_5 < price <= fib_786:  # It seems like 'fib_5' is undefined here. Perhaps it should be 'fib_618'?
        print(f"The price {price} is within the range 0.5 - 0.786 ({fib_5}, {fib_786})")
    else:
        print(f"The price {price} is outside the range ({min_val}, {max_val})")

def possible_retracement_v2(price, min_val, max_val):
    fib_5, fib_618, fib_786 = fibonacci_levels_v2(min_val, max_val)
    if fib_618 < price <= fib_786:
        #print(f"The price {price} is within the range of 0.618 - 0.786 ({fib_618}, {fib_786})")
        return True
    elif fib_5 < price <= fib_786:  # It seems like 'fib_5' is undefined here. Perhaps it should be 'fib_618'?
        #print(f"The price {price} is within the range 0.5 - 0.786 ({fib_5}, {fib_786})")
        return True
    else:
        #print(f"The price {price} is outside the range ({min_val}, {max_val})")
        return False

get_min_max_data()
print(f"Min Max VALUES: {min_max_vals}")
print(f"Min Max TIMES: {min_max_time}")
print(f"Min Max TYPES: {min_max_type}")

In [ ]:
def get_pivot_idx():
    i = 0
    pivot = min_max_vals[0]
    min_val = min_max_vals[0]
    while(i < len(min_max_vals)):
        if(min_max_vals[i] < min_val):
            return i
        i += 1

bullish = min_max_vals[:get_pivot_idx()]
bullish_time = min_max_time[:get_pivot_idx()]

trend = np.polyfit(bullish_time, bullish, 1)
# plot the scatter points
plt.plot(bullish_time, bullish, '*')
# calculate the trendline
trendpoly = np.poly1d(trend)
# plot the trend line
plt.plot(bullish_time, trendpoly(bullish_time))

In [ ]:
mins = []
for i in range(0, len(bullish)):
    if(trendpoly(bullish_time[i]) > bullish[i]):
        mins.append(bullish[i])

print(mins)
i = 0
min_val = float("inf")
while (i + 1 < len(mins)):
    min_val = mins[i]
    next_min_val = mins[i+1]
    range_to_review = -1
    for x in range(len(bullish)):
        if bullish[x] == next_min_val:
            range_to_review = x
    max_val = 0
    for j in range(i, range_to_review):
        if(trendpoly(bullish_time[j]) < bullish[j]):
            max_val = max(max_val, bullish[j])
    print(f"Min val: {min_val}, next: {next_min_val}, max: {max_val}")
    if min_val < next_min_val < max_val:
        possible_retracement(next_min_val, min_val, max_val)
    i += 1